# Operations problems with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports the construction and solution of optimal power system
scheduling problems (Operations Problems). Operations problems form the fundamental
building blocks for [sequential simulations](../../notebook/PowerSimulations_examples/sequential_simulations.ipynb).
This example shows how to specify and customize a the mathematics that will be applied to the data with
an `OperationsProblemTemplate`, build and execute an `OperationsProblem`, and access the results.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/grid-mod/rts-gmlc) dataset. Let's
download and extract the data.

In [5]:
rts_dir = SIIPExamples.download("https://github.com/GridMod/RTS-GMLC")
rts_src_dir = joinpath(rts_dir, "RTS_Data", "SourceData")
rts_siip_dir = joinpath(rts_dir, "RTS_Data", "FormattedData", "SIIP");

### Create a `System` from RTS-GMLC data just like we did in the [parsing tabular data example.](../../notebook/PowerSystems_examples/parse_tabulardata.jl)

In [6]:
rawsys = PowerSystems.PowerSystemTableData(
    rts_src_dir,
    100.0,
    joinpath(rts_siip_dir, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(rts_siip_dir, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(rts_siip_dir, "generator_mapping.yaml"),
);

sys = System(rawsys; forecast_resolution = Dates.Hour(1));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/s

## Define a problem specification with an `OpModelTemplate`
The `DeviceModel` constructor is to create an assignment between PowerSystems device types
and the subtypes of `AbstractDeviceFormulation`. PowerSimulations has a variety of different
`AbstractDeviceFormulation` subtypes that can be applied to different PowerSystems device types,
each dispatching to different methods for populating optimization problem objectives, variables,
and constraints.

In [7]:
TypeTree(PSI.AbstractDeviceFormulation, scopesep="\n")

PowerSimulations
AbstractDeviceFormulation
├──PowerSimulations
│  AbstractThermalFormulation
│  ├──PowerSimulations
│  │  AbstractThermalDispatchFormulation
│  │  ├──ThermalDispatchNoMin (0 children)
│  │  ├──ThermalDispatch (0 children)
│  │  └──ThermalRampLimited (0 children)
│  └──PowerSimulations
│     AbstractThermalUnitCommitment
│     ├──ThermalMultiStartUnitCommitment (0 children)
│     ├──ThermalBasicUnitCommitment (0 children)
│     └──ThermalStandardUnitCommitment (0 children)
├──PowerSimulations
│  AbstractBranchFormulation
│  ├──PowerSimulations
│  │  AbstractDCLineFormulation
│  │  ├──HVDCUnbounded (0 children)
│  │  ├──HVDCDispatch (0 children)
│  │  ├──HVDCLossless (0 children)
│  │  └──PowerSimulations
│  │     VoltageSourceDC (0 children)
│  ├──PowerSimulations
│  │  AbstractBoundedBranchFormulation
│  │  ├──StaticLineBounds (0 children)
│  │  └──StaticTransformerBounds (0 children)
│  ├──StaticLineUnbounded
│  ├──StaticLine
│  ├──StaticTransformer
│  ├──StaticTransformerUnbounded
│  └──FlowMonitoredLine
├──PowerSimulations
│  AbstractLoadFormulation
│  ├──PowerSimulations
│  │  AbstractControllablePowerLoadFormulation
│  │  ├──DispatchablePowerLoad (0 children)
│  │  └──InterruptiblePowerLoad (0 children)
│  └──StaticPowerLoad
├──PowerSimulations
│  AbstractRenewableFormulation
│  └──PowerSimulations
│     AbstractRenewableDispatchFormulation
│     ├──RenewableConstantPowerFactor (0 children)
│     └──RenewableFullDispatch (0 children)
├──FixedOutput
├──PowerSimulations
│  AbstractRegulationFormulation
│  ├──DeviceLimitedRegulation
│  └──ReserveLimitedRegulation
├──PowerSimulations
│  AbstractStorageFormulation
│  ├──BookKeeping
│  └──BookKeepingwReservation
└──PowerSimulations
   AbstractHydroFormulation
   ├──PowerSimulations
   │  AbstractHydroDispatchFormulation
   │  ├──PowerSimulations
   │  │  AbstractHydroReservoirFormulation (2 children)
   │  └──HydroDispatchRunOfRiver (0 children)
   └──PowerSimulations
      AbstractHydroUnitCommitment

### Branch Formulations
Here is an example of relatively standard branch formulations. Other formulations allow
for selective enforcement of transmission limits and greater control on transformer settings.

In [8]:
branches = Dict{Symbol, DeviceModel}(
    :L => DeviceModel(Line, StaticLine),
    :T => DeviceModel(Transformer2W, StaticTransformer),
    :TT => DeviceModel(TapTransformer, StaticTransformer),
)

Dict{Symbol,DeviceModel} with 3 entries:
  :T  => DeviceModel{Transformer2W,StaticTransformer}(Transformer2W, StaticTran…
  :TT => DeviceModel{TapTransformer,StaticTransformer}(TapTransformer, StaticTr…
  :L  => DeviceModel{Line,StaticLine}(Line, StaticLine, nothing, ServiceModel[])

### Injection Device Formulations
Here we define dictionary entries for all devices that inject or withdraw power on the
network. For each device type, we can define a distinct `AbstractDeviceFormulation`. In
this case, we're defining a basic unit commitment model for thermal generators,
curtailable renewable generators, and fixed dispatch (net-load reduction) formulations
for `HydroFix` and `RenewableFix` devices. Additionally, we've enabled a simple load
shedding demand response formulation for `InterruptableLoad` devices.

In [9]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
    :ILoads => DeviceModel(InterruptibleLoad, InterruptiblePowerLoad),
)

Dict{Symbol,DeviceModel} with 7 entries:
  :ILoads     => DeviceModel{InterruptibleLoad,InterruptiblePowerLoad}(Interrup…
  :HydroROR   => DeviceModel{HydroDispatch,FixedOutput}(HydroDispatch, FixedOut…
  :Generators => DeviceModel{ThermalStandard,ThermalStandardUnitCommitment}(The…
  :Ren        => DeviceModel{RenewableDispatch,RenewableFullDispatch}(Renewable…
  :Hydro      => DeviceModel{HydroEnergyReservoir,HydroDispatchRunOfRiver}(Hydr…
  :Loads      => DeviceModel{PowerLoad,StaticPowerLoad}(PowerLoad, StaticPowerL…
  :RenFx      => DeviceModel{RenewableFix,FixedOutput}(RenewableFix, FixedOutpu…

### Service Formulations
We have two `VariableReserve` types, parameterized by their direction. So, similar to
creating `DeviceModel`s, we can create `ServiceModel`s. The primary difference being
that `DeviceModel` objects define how constraints get created, while `ServiceModel` objects
define how constraints get modified.

In [10]:
services = Dict(
    :ReserveUp => ServiceModel(VariableReserve{ReserveUp}, RangeReserve),
    :ReserveDown => ServiceModel(VariableReserve{ReserveDown}, RangeReserve),
)

Dict{Symbol,ServiceModel{D,RangeReserve} where D<:Service} with 2 entries:
  :ReserveDown => ServiceModel{VariableReserve{ReserveDown},RangeReserve}(Varia…
  :ReserveUp   => ServiceModel{VariableReserve{ReserveUp},RangeReserve}(Variabl…

### Wrap it up into an `OperationsProblemTemplate`

In [11]:
template_uc = OperationsProblemTemplate(CopperPlatePowerModel, devices, branches, services);

## `OperationsProblem`
Now that we have a `System` and an `OperationsProblemTemplate`, we can put the two together
to create an `OperationsProblem` that we solve.

### Optimizer
It's most convenient to define an optimizer instance upfront and pass it into the
`OperationsProblem` constructor. For this example, we can use the free Cbc solver with a
relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [12]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Build an `OperationsProblem`
The construction of an `OperationsProblem` essentially applies an `OperationsProblemTemplate`
to `System` data to create a JuMP model.

In [13]:
op_problem =
    OperationsProblem(GenericOpProblem, template_uc, sys; optimizer = solver, horizon = 12)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/base.jl:353
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/devices/thermal_generation.jl:612



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
  ser

The principal component of the `OperationsProblem` is the JuMP model. For small problems,
you can inspect it by simply printing it to the screen:
```julia
op_problem.psi_container.JuMPmodel
```

For anything of reasonable size, that will be unmanageable. But you can print to a file:
```julia
f = open("testmodel.txt","w"); print(f,op_problem.psi_container.JuMPmodel); close(f)
```

In addition to the JuMP model, an `OperationsProblem` keeps track of a bunch of metadata
about the problem and some references to pretty names for constraints and variables.
All of these details are contained within the `psi_container` field.

In [14]:
print_struct(typeof(op_problem.psi_container))

mutable struct PowerSimulations.PSIContainer
    JuMPmodel::Union{Nothing, JuMP.AbstractModel}
    time_steps::UnitRange{Int64}
    resolution::TimePeriod
    settings::PowerSimulations.PSISettings
    settings_copy::PowerSimulations.PSISettings
    variables::Dict{Symbol,AbstractArray}
    constraints::Dict{Symbol,AbstractArray}
    cost_function::JuMP.AbstractJuMPScalar
    expressions::Dict{Symbol,JuMP.Containers.DenseAxisArray}
    parameters::Union{Nothing, Dict{Symbol,PowerSimulations.ParameterContainer}}
    initial_conditions::PowerSimulations.InitialConditions
    pm::Union{Nothing, PowerModels.AbstractPowerModel}
end


### Solve an `OperationsProblem`

In [15]:
res = solve!(op_problem);

## Results Inspection
PowerSimulations collects the `OperationsProblem` results into a struct:

In [16]:
print_struct(PSI.SimulationResults)

 struct PowerSimulations.SimulationResults
    base_power::Float64
    variable_values::Dict{Symbol,DataFrame}
    total_cost::Dict
    optimizer_log::Dict
    time_stamp::DataFrame
    dual_values::Dict{Symbol,Any}
    results_folder::Union{Nothing, String}
    parameter_values::Dict{Symbol,DataFrame}
end


### Optimizer Log
The optimizer summary is included

In [17]:
get_optimizer_log(res)

Dict{Symbol,Any} with 9 entries:
  :timed_solve_time   => 15.0073
  :solve_bytes_alloc  => 1683272021
  :solve_time         => 1.57828
  :obj_value          => 313118.0
  :solver             => "COIN Branch-and-Cut (Cbc)"
  :sec_in_gc          => 0.885816
  :dual_status        => NO_SOLUTION
  :primal_status      => FEASIBLE_POINT
  :termination_status => OPTIMAL

### Total Cost (objective function value)

In [18]:
get_total_cost(res)

Dict{Symbol,Float64} with 1 entry:
  :OBJECTIVE_FUNCTION => 313118.0

### Variable Values
The solution value data frames for variable in the `op_problem.psi_container.variables`
dictionary is stored:

In [19]:
variable_values = get_variables(res)

Dict{Symbol,DataFrame} with 13 entries:
  :P__ThermalStandard       => 12×76 DataFrame. Omitted printing of 70 columns…
  :P__RenewableDispatch     => 12×30 DataFrame. Omitted printing of 24 columns…
  :Reg_Down__VariableReser… => 12×102 DataFrame. Omitted printing of 96 columns…
  :P__HydroEnergyReservoir  => 12×19 DataFrame. Omitted printing of 14 columns…
  :Flex_Down__VariableRese… => 12×102 DataFrame. Omitted printing of 96 columns…
  :Reg_Up__VariableReserve… => 12×102 DataFrame. Omitted printing of 96 columns…
  :Spin_Up_R2__VariableRes… => 12×25 DataFrame. Omitted printing of 19 columns…
  :On__ThermalStandard      => 12×76 DataFrame. Omitted printing of 70 columns…
  :Flex_Up__VariableReserv… => 12×102 DataFrame. Omitted printing of 96 columns…
  :Start__ThermalStandard   => 12×76 DataFrame. Omitted printing of 70 columns…
  :Stop__ThermalStandard    => 12×76 DataFrame. Omitted printing of 70 columns…
  :Spin_Up_R1__VariableRes… => 12×34 DataFrame. Omitted printing of 28 colum

Note that the time stamps are missing from the dataframes in `variable_values`...

The time stamps for each value in the time series used in the `OperationsProblem` is
included separately from the variable value results.

In [20]:
get_time_stamp(res)

,Range
,DateTime
1,2020-01-01T00:00:00
2,2020-01-01T01:00:00
3,2020-01-01T02:00:00
4,2020-01-01T03:00:00
5,2020-01-01T04:00:00
6,2020-01-01T05:00:00
7,2020-01-01T06:00:00
8,2020-01-01T07:00:00
9,2020-01-01T08:00:00


## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*